In [16]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
from shared.models import SAT1Start, ShallowConvNet, EEGNet
from shared.generators import SAT1DataGenerator, NewSAT1DataGenerator
from shared.utilities import earlyStopping_cb
from sklearn.metrics import classification_report
import random
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [17]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [26]:
participants = data.participant.values.tolist()
test_participants = random.sample(participants, 5)
train_participants = [p for p in participants if p not in test_participants]

In [27]:
test_data = data.sel(participant=test_participants)
train_data = data.sel(participant=train_participants)

# train_data = data.sel(
#     participant=[
#         "0021",
#         "0022",
#         "0023",
#         "0024",
#     ]
# )

# test_data = data.sel(
#     participant=[
#         "0025"
#     ]
# )

In [28]:
batch_size = 16
train_gen = NewSAT1DataGenerator(train_data, batch_size=batch_size)
test_gen = NewSAT1DataGenerator(test_data, batch_size=batch_size)

# tf.config.optimizer.set_experimental_options({"layout_optimizer": False})

In [45]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model

In [46]:
model = SAT1Start(30, 157, 4)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30, 157, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 30, 153, 16)       96        
                                                                 
 dropout (Dropout)           (None, 30, 153, 16)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 149, 32)       12832     
                                                                 
 dropout_1 (Dropout)         (None, 26, 149, 32)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 22, 149, 64)       10304     
                                                                 
 batch_normalization (Batch  (None, 22, 149, 64)       256   

In [47]:
model.fit(
    train_gen,
    epochs=20,
    # TODO: Create val_gen
    validation_data=test_gen,
    use_multiprocessing=True,
    workers=8,
)

# model.fit(
#     train_gen,
#     epochs=20,
#     # TODO: Create val_gen (use 6th participant?)
#     validation_data=test_gen,
#     use_multiprocessing=True,
#     workers=8,
# )

Epoch 1/20


2023-08-23 13:04:00.993199: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


783/783 [==============================] - 13s 15ms/step - loss: 1.3141 - accuracy: 0.4184 - val_loss: 1.3827 - val_accuracy: 0.4993
Epoch 2/20
783/783 [==============================] - 11s 14ms/step - loss: 0.7947 - accuracy: 0.6740 - val_loss: 0.6465 - val_accuracy: 0.7389
Epoch 3/20
783/783 [==============================] - 11s 14ms/step - loss: 0.6377 - accuracy: 0.7409 - val_loss: 1.4598 - val_accuracy: 0.6191
Epoch 4/20
783/783 [==============================] - 11s 14ms/step - loss: 0.5801 - accuracy: 0.7677 - val_loss: 0.8980 - val_accuracy: 0.6584
Epoch 5/20
783/783 [==============================] - 11s 14ms/step - loss: 0.5538 - accuracy: 0.7818 - val_loss: 2.0849 - val_accuracy: 0.5046
Epoch 6/20
783/783 [==============================] - 11s 14ms/step - loss: 0.5350 - accuracy: 0.7863 - val_loss: 2.4800 - val_accuracy: 0.4810
Epoch 7/20
783/783 [==============================] - 11s 14ms/step - loss: 0.5173 - accuracy: 0.7955 - val_loss: 2.3125 - val_accuracy: 0.4562
Epo